In [1]:
import pandas as pd
from datetime import datetime
matches_2022 = pd.read_csv('data/matches_2022.csv')
matches_2022.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2022-8888,Atp Cup,Hard,16,A,20220103,300,200000,NaN,NaN,...,50.0,32.0,7.0,10.0,3.0,5.0,11.0,3308.0,19.0,2260.0
1,2022-8888,Atp Cup,Hard,16,A,20220103,299,133430,NaN,NaN,...,33.0,21.0,8.0,9.0,3.0,6.0,14.0,2475.0,20.0,2230.0
2,2022-8888,Atp Cup,Hard,16,A,20220103,298,105138,NaN,NaN,...,80.0,62.0,20.0,16.0,6.0,7.0,19.0,2260.0,9.0,3706.0
3,2022-8888,Atp Cup,Hard,16,A,20220103,297,105807,NaN,NaN,...,27.0,17.0,1.0,7.0,4.0,8.0,20.0,2230.0,860.0,18.0
4,2022-8888,Atp Cup,Hard,16,A,20220103,296,106421,NaN,NaN,...,35.0,22.0,4.0,8.0,3.0,7.0,2.0,8640.0,11.0,3308.0


In [2]:
# Get only the column names which include the substring 'name'
matches_2022.filter(regex='name').columns

Index(['tourney_name', 'winner_name', 'loser_name'], dtype='object')

In [3]:
# Get only rows where the value of column 'winner_name' or 'loser_name' includes 'Tim van Rijthoven' without accounting for case
matches_2022[matches_2022['winner_name'].str.contains('Tim van Rijthoven', case=False) | matches_2022['loser_name'].str.contains('Tim van Rijthoven', case=False)]

# Same as above but only show the winner name or loser name columns
matches_2022[matches_2022['winner_name'].str.contains('Tim van Rijthoven', case=False) | matches_2022['loser_name'].str.contains('Tim van Rijthoven', case=False)][['winner_name', 'loser_name']]

,winner_name,loser_name
1374,Tim Van Rijthoven,Daniil Medvedev
1376,Tim Van Rijthoven,Felix Auger Aliassime
1379,Tim Van Rijthoven,Hugo Gaston
1386,Tim Van Rijthoven,Taylor Fritz
1397,Tim Van Rijthoven,Matthew Ebden
1550,Tim Van Rijthoven,Federico Delbonis
1611,Tim Van Rijthoven,Reilly Opelka
1641,Tim Van Rijthoven,Nikoloz Basilashvili
1656,Novak Djokovic,Tim Van Rijthoven
1721,Mitchell Krueger,Tim Van Rijthoven


In [12]:
# Turn the above 2 lines into a function which outputs the index number of the found rows and the ratio of Tim's presence in the winner_name to loser_name columns
def find_matches_by_name(name):
    # Find the rows where name is present in the winner_name or loser_name columns
    name_matches_2022 = matches_2022[matches_2022['winner_name'].str.contains(name, case=False) | matches_2022['loser_name'].str.contains(name, case=False)]
    name_matches_index = name_matches_2022.index

    # Get the percent wins of the player
    won_percent = len(name_matches_2022[name_matches_2022['winner_name'].str.contains(name, case=False)]) / len(name_matches_2022)

    # From the latest row in the row index, get the value of the largest date value in the tourney_date column
    name_matches_latest_date = name_matches_2022.iloc[-1]['tourney_date']
    name_matches_latest_date = datetime.strptime(str(name_matches_latest_date), '%Y%m%d').strftime('%Y-%m-%d') # YYYY-MM-DD format

    # If the winner_name in the last row is equal to the name (not accounting for case), get that row's winner_id, otherwise get the loser_id
    if name_matches_2022.iloc[-1]['winner_name'].lower() == name.lower():
        id = name_matches_2022.iloc[-1]['winner_id']
    else:
        id = name_matches_2022.iloc[-1]['loser_id']

    # As a dictionary, return the name of the player, a list of the index numbers of the rows where the player is present and the ratio of the player's presence in the winner_name to loser_name columns
    return {'name': name, 'id': id, 'matches_2022': [name_matches_index], 'win_percent': won_percent, 'latest_date': name_matches_latest_date}

# Get the index number of the rows where Tim is present and the ratio of Tim's presence in the winner_name to loser_name columns
find_matches_by_name('Tim van Rijthoven')


{'name': 'Tim van Rijthoven',
 'id': 126646,
 'matches_2022': [Index([1374, 1376, 1379, 1386, 1397, 1550, 1611, 1641, 1656, 1721, 2125, 2174,
         2331, 2452, 2554],
        dtype='int64')],
 'win_percent': 0.6,
 'latest_date': '2022-10-17'}

In [14]:
def find_matches_by_id(id):
    # Find the rows where name is present in the winner_name or loser_name columns
    id_won_matches_2022 = matches_2022[matches_2022['winner_id'] == id]
    id_lost_matches_2022 = matches_2022[matches_2022['loser_id'] == id]
    id_matches_2022 = pd.concat([id_won_matches_2022, id_lost_matches_2022])
    
    id_matches_index = id_matches_2022.index

    # Get the percent wins of the player
    won_percent = len(id_won_matches_2022) / len(id_matches_2022)
    
    # From the latest row in the row index, get the value of the largest date value in the tourney_date column
    id_matches_latest_date = id_matches_2022.iloc[-1]['tourney_date']
    id_matches_latest_date = datetime.strptime(str(id_matches_latest_date), '%Y%m%d').strftime('%Y-%m-%d') # YYYY-MM-DD format

    # Get the name of the player
    name = id_matches_2022.iloc[-1]['winner_name'] if id_matches_2022.iloc[-1]['winner_id'] == id else id_matches_2022.iloc[-1]['loser_name']

    # As a dictionary, return the name of the player, a list of the index numbers of the rows where the player is present and the ratio of the player's presence in the winner_name to loser_name columns
    return {'name': name, 'matches_2022': [id_matches_index], 'win_percent': won_percent, 'latest_date': id_matches_latest_date}

find_matches_by_id(126646)

{'name': 'Tim Van Rijthoven',
 'matches_2022': [Index([1374, 1376, 1379, 1386, 1397, 1550, 1611, 1641, 2125, 1656, 1721, 2174,
         2331, 2452, 2554],
        dtype='int64')],
 'win_percent': 0.6,
 'latest_date': '2022-10-17'}

In [15]:
# Get the unique ids of the players
unique_ids = set(list(matches_2022['winner_id'].unique()) + list(matches_2022['loser_id'].unique()))

# Now make this performance even better by using multiprocessing
import multiprocessing as mp

pool = mp.Pool(processes=4)
all_people = pool.map(find_matches_by_id, unique_ids)
pool.close()

# Turn the list of dictionaries into a dataframe
all_people_df = pd.DataFrame(all_people)

# # Sort the dataframe by the win/lose ratio
# all_matches_matches_2022.sort_values(by='win_lose', ascending=False)

# # Get the top 10 players with the highest win/lose ratio
# all_matches_matches_2022.sort_values(by='win_lose', ascending=False).head(10)

all_people_df

,name,matches_2022,win_percent,latest_date
0,Nicolas Mejia,"[[2824, 2808, 2810]]",0.333333,2022-03-04
1,Dennis Novak,"[[1594, 2778, 366, 561, 1633, 2623, 2780]]",0.285714,2022-03-04
2,Gilles Simon,"[[342, 1303, 1339, 1389, 2271, 2645, 2665, 330...",0.466667,2022-10-31
3,Kasidit Samrej,"[[2868, 2870]]",1.000000,2022-09-16
4,Brandon Nakashima,"[[146, 157, 448, 480, 740, 832, 1001, 1265, 13...",0.603448,2022-10-31
...,...,...,...,...
403,J J Wolf,"[[588, 718, 818, 862, 1899, 1909, 1925, 2102, ...",0.535714,2022-10-24
404,Sergey Fomin,"[[2845, 2846, 2861, 2863]]",0.500000,2022-03-04
405,Rio Noguchi,"[[2424, 2409]]",0.500000,2022-10-03
406,Soon Woo Kwon,"[[74, 128, 184, 335, 426, 501, 623, 989, 1054,...",0.454545,2022-09-15


In [ ]:
# TODO: import all the csv files from the past 10 years and then do the same thing as above. Can just add the new csv file names to the functions above and then run the functions again.
